<a href="https://colab.research.google.com/github/mayurijoshi106/mayuri_INFO5731_spring2020/blob/master/INFO5731_Assignment_Three_MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [199]:
import pandas as pd
review_df = pd.read_csv('/content/amazonreviews.csv')

#1.1 Count the frequency of all the N-grams (N=3).

import nltk
nltk.download('punkt')
from textblob import TextBlob

def get_ngram(num):
  ngram_dict = {}

  for review in review_df['CleanData']:
    ngram_list = TextBlob(review).ngrams(num)
    if(len(ngram_list) > 0):
      for ngram in ngram_list:
        ngram_key = tuple(ngram)
        if ngram_key in ngram_dict:
          ngram_dict[ngram_key] += 1
        else:
          ngram_dict[ngram_key] = 1
  return(ngram_dict)

ngram_num = 3
trigram_dict = get_ngram(ngram_num)
trigram_df = pd.DataFrame.from_dict(trigram_dict, orient='index', columns=['Tri-grams          ' + 'Frequency'])
print(trigram_df)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
                       Tri-grams          Frequency
(give, sister, cook)                              1
(sister, cook, clean)                             1
(cook, clean, take)                               1
(clean, take, trash)                              1
(take, trash, love)                               1
...                                             ...
(brand, new, pretti)                              1
(new, pretti, much)                               1
(pretti, much, give)                              1
(much, give, dell)                                1
(give, dell, point)                               1

[339 rows x 1 columns]


In [254]:
#1.2 Calculate the probabilities for all the bigrams in 
#the dataset by using the fomular count(w2 w1) / count(w2). 
#For example, count(really like) / count(really) = 1 / 3 = 0.33.

bigram_dict = get_ngram(2)
unigram_dict = get_ngram(1)

bigram_prob = {}
for bigram in bigram_dict:
  word = tuple((bigram[0],))
  count_bi = bigram_dict[bigram]
  count_uni = unigram_dict[word]
  prob = count_bi / count_uni
  bigram_prob[bigram] = prob

bigram_df = pd.DataFrame.from_dict(bigram_prob, orient='index', columns=['Bi-grams Probability'])
print(bigram_df)

                Bi-grams Probability
(give, sister)              0.333333
(sister, cook)              1.000000
(cook, clean)               1.000000
(clean, take)               1.000000
(take, trash)               0.500000
...                              ...
(new, pretti)               0.200000
(pretti, much)              0.500000
(much, give)                1.000000
(give, dell)                0.333333
(dell, point)               0.333333

[351 rows x 1 columns]


In [203]:
#1.3
import nltk
import math
nltk.download('brown')

dfreq = {}
for review in review_df['CleanData']:
  nps = (TextBlob(review)).noun_phrases
  for np in nps:
    if np in dfreq:
      dfreq[np] += 1
    else:
      dfreq[np] = 1
max_freq = max({freq for freq in dfreq.values()})

all_np_prob = []
for review in review_df['CleanData']:
  each_row_pb = {}
  each_nps = (TextBlob(review)).noun_phrases
  for each_np in each_nps:
    if each_np in each_row_pb:
      each_row_pb[each_np] += 1
    else:
      each_row_pb[each_np] = 1
  for k, v in each_row_pb.items():
    value = v/max_freq
    each_row_pb[k] = value
  all_np_prob.append(each_row_pb)

row_index = [n for n in range(1, len(all_np_prob)+1)]

prob_df = pd.DataFrame (all_np_prob, index=row_index)
prob_df.fillna(0, inplace=True)
prob_df

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


,sister cook,new laptop,laptop spec laptop gobb memori,hard drive space,pictur hope,im pretti,week use,lot time research,check box speed memori,sound screen,great happi choic use person busi use,sure time touch screen worthi,extra cost,thing dont,light continu press,charact key,new recurv use,overal good laptop casual use fan backlit keyboard,key sometim,laptop anyon look,fast easi use solut shop,laptop use wife replac,dell inspiron laptop,keyboard touchscreen display havent,complaint prais,non techi persongood memori,lit screen easi use use laptopfor email publish work,laptop isheavi,carri laptopoften,desk athom,price good capabilitiesit,work scratch outsid use im greatli need good comput,work terrif,fast touch screen cours doesnt realli use,lazi use mous comput,great plea amazon dell,new comput,scratch top,new buyer bewar,product arriv damag work seller refus fix replac blame damag bewar purchas,brown box seller,satisfi purchas,good laptop high,happi product,fast perfect comput,great product,load ram sata drive,digit password,boom load backlit god,super slim lightweight laptop troubl,cach chrome noth,page load touch,whole thing,problem anymor,thing run background,open set click privaci click background app choos app run background section,toggl switch app,thing sometm hour day joy happi seller,mine costco,super easi return fan louder airplan,overlook didnt,time comput,soooo sloooowwww year,old comput kid run program load page,piec junk brand,new pretti,dell point
1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [225]:
#2.1 documents-terms weights (tf*idf) matrix bold text.
from sklearn.feature_extraction.text import TfidfVectorizer

df2 = []
for sent in review_df['CleanData']:
  df2.append(sent)

vectorizer = TfidfVectorizer(input=corpus, min_df=0.1, stop_words='english')
response = vectorizer.fit_transform(df2)
tf_idf_df=pd.DataFrame(response.T.todense(), vectorizer.get_feature_names())
tf_idf_df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
backlit,0.000000,0.000000,0.000000,0.000000,0.000000,0.313544,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.156223,0.000000
best,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.847208,0.000000,0.000000,0.0,0.156223,0.000000
bewar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176819,0.324486,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
bought,0.259494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.233235
box,0.000000,0.263645,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.324486,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
brand,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.353639,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.233235
comput,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.478656,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.784881,0.000000,0.0,0.000000,0.420917
dell,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.350012,0.000000,0.159552,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.210458
difficult,0.000000,0.000000,0.000000,0.421508,0.000000,0.313544,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
drive,0.259494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.156223,0.000000


In [229]:
#2.2 To rank the documents with respect to query 
#(design a query by yourself, for example, "An Outstanding movie 
#with a haunting performance and best character development") by using cosine similarity.

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
clean_query = ''
query = 'The Inspiron 15 (3558) is a fully functional, well-made laptop with good speed and a low price point.'
regex = re.compile('[^a-zA-Z\s]')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

#numbers, punctuation and special characters
def punt_spl_numbers(x):
    return re.sub(regex, '', x)

#stop words
def stop_words(x):
  nostop_words = " ".join([word for word in x.split() if word not in stopwords.words('english')])
  return nostop_words

#stemming
def get_stemming(x):
  stem = " ".join([stemmer.stem(i) for i in x.split()])
  return stem

#lemmitization
def get_lemmatization(x):
  lem = " ".join([lemmatizer.lemmatize(i) for i in x.split()])
  return lem

clean_query = punt_spl_numbers(query)
clean_query = clean_query.lower()
clean_query = stop_words(clean_query)
clean_query = get_stemming(clean_query)
clean_query = get_lemmatization(clean_query)
print('Cleaned Query: ' + clean_query)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Cleaned Query: inspiron fulli function wellmad laptop good speed low price point


In [244]:
from sklearn.feature_extraction.text import TfidfVectorizer

df2 = []
for sent in review_df['CleanData']:
  df2.append(sent)

vectorizer = TfidfVectorizer(input=corpus, min_df=0.1, stop_words='english')
response = vectorizer.fit_transform(df2)
tf_idf_df=pd.DataFrame(response.T.todense(), vectorizer.get_feature_names())
tf_idf_df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
backlit,0.000000,0.000000,0.000000,0.000000,0.000000,0.313544,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.156223,0.000000
best,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.847208,0.000000,0.000000,0.0,0.156223,0.000000
bewar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176819,0.324486,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
bought,0.259494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.233235
box,0.000000,0.263645,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.324486,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
brand,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.353639,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.233235
comput,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.478656,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.784881,0.000000,0.0,0.000000,0.420917
dell,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.350012,0.000000,0.159552,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.210458
difficult,0.000000,0.000000,0.000000,0.421508,0.000000,0.313544,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
drive,0.259494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.156223,0.000000


In [253]:
# Calculate cosine similarity of the query to others documents by using SKLEARN
import pandas
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


df2 = []
for sent in review_df['CleanData']:
  df2.append(sent)

vectorizer = TfidfVectorizer(input=corpus, min_df=0.1, stop_words='english')
response = vectorizer.fit_transform(df2)
cosinedct = {}
for i in range(0, response.shape[0]):
  cosim=cosine_similarity(response[10], response[i])
  cosinedct[str(i)] = cosim[0][0]
  i+=1

sorted_keys = (sorted(cosinedct, key=cosinedct.get,reverse=True))
sorted_cosinedct = {}
for key in sorted_keys:
  sorted_cosinedct[key] = cosinedct[key];

rw = []
cosine = []
for skey in sorted_cosinedct:
  rw.append(df2[int(skey)])
  cosine.append(sorted_cosinedct[skey])

rankdf=pandas.DataFrame.from_dict (rw,orient = 'index', columns = ['ranking'])
rankdf ['cosine similarity'] = cosine
rankdf.head(20)

AttributeError: ignored

# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: https://github.com/mayurijoshi106/mayuri_INFO5731_spring2020/blob/master/amazonreviews_sentiment.csv